# **Motivation and Goals**

The The National Electrical Code ([NEC](https://www.nfpa.org/codes-and-standards/all-codes-and-standards/list-of-codes-and-standards/detail?code=70)) mandates and describes how to install charge points and Electric Vehicle Supply Equipment (EVSE). Section **625.42** describes how to rate the load of EV charging. This is relevant if you plan to install more chargers than technically possible; “Electric vehicle charging loads shall be considered to be continuous loads.”, and therefore you need to follow the 80% rule. This states that continuous load must be 20% below the breaker capacity. On the other hand, Section **625.42** also states: “Where an automatic load management system is used, the maximum equipment load on a service and feeder shall be the maximum load permitted by the automatic load management system.”
In other words, when we want to run charging stations above 80% of line capacity, you must have an intelligent load management system to maintain control of the loads. This load management system sets limits on the charge point (EVSE) and thereby ensures the 80% rule. You can now oversubscribe breakers. The good news is that power management software such as EViDec® can be set to intelligently optimally manage the power at a circuit, panel, site, or transformer level. 
Moreover, adding EVSEs to an electrical system, needs at least a 30-day load study to protect the electrical system from potential system overload. The NEC Code **220.87** specifies that the calculation of a feeder or service load for existing installations shall be permitted to use actual maximum demand to determine the existing load under all of the following conditions:
* 1)	The maximum demand data is available for a 1-year period.
* 2)	The maximum demand at 125% plus the new load does not exceed the ampacity of the feeder or rating of the service.
* 3)	The feeder has overcurrent protection in accordance with **240.4**, and the service has overload protection in accordance with 230.90.

We at [SWTCH](https://swtchenergy.com/) took the [RTEM](https://www.rtemhackathon.com/) data as an opportunity to:
* 1)	Provide a simple and consolidate approach to monitor the existing load using a data analytics approach,
* 2)	Provide a load management system to maximize utilizing the existing infrastructure while minimizing the installation and operation costs for potential customers,
* 3)	Use other available data source to support our proposed solution and propose a more realistic metric.


# **Data Model**
Study of an existing load behavior for a long period can be tricky, specially in presence of 1) different building applications, 2) dispersity in the load profiles, and 3) measurement outliers. We deployed a k-means approach to categorize the daily load profile, and to distinguish the load profiles in the different periods of a year. 
Using k-means we would make sure there is an obvious distinguish between different daily load profile of a year, and then perform an energy study from each individual cluster. Thus by performing a partitioning k-means clustering method, the profile shape of each building can be studied specifically. The k-means is an unsupervised learning algorithm for clustering a training set ${x_1,...,x_N}$ consisting of $N$ observations of a random D-dimensional Euclidean variable $x$ into $K$ groups of equal prototype. The prototype can be defined as D-dimensional vectors of centroids $\mu_k$ associated with the $k^{th}$ cluster, where $k=1,…,K$. The data point assignment $\alpha$ is a set of binary indicator variables $\alpha_{nk}∈ \{0,1\}$, indicates assigning of $x_n$ to the $k^{th}$ cluster, so if data point $x_n$ is assigned to cluster k then $α_{nk}=1$, otherwise $α_{n k}=0$.  
By defining the cost function by sum of squared distances between each data point $x_n$ to its assigned centroids vector $\mu_k$; 

$$J=\sum_{n=1}^N\sum_{k=1}^K \alpha_{nk⁡}(‖x_n-μ_k‖^2)$$

The goal is to minimize the distortion measure $J$ through finding i) the assignment set $\{α_{nk}\}$, and ii) the set of $\{μ_k\}$ which minimize $J$. To deploy this two-stage optimization, it is needed to initiate it with some values for the $μ_k$, and then minimize J with respect to the $α_{nk}$, while $\mu_k$ is held fix. It can be done simply through assigning the data point to the closest cluster center with;

$$
\left\{\begin{matrix}
1 & k = \underset{j}{Argmin}\left\| x_n - \mu_k \right\| \\
0 & \textrm{otherwise}\\
\end{matrix}\right.
$$

and secondly, we hold $\alpha_{nk}$ and minimize J with respect to the $μ_k$ by setting $\partial J/ \partial \mu_k=0$ which yields to;

$$
\mu_k=\frac{\sum_{n=1}^N \alpha_{nk}x_n}{\sum_{n=1}^N \alpha_{nk}},
$$

where the second equation assigns each sample to its nearest centroid and the third creates new centroids by taking the mean value of all of the samples assigned to the previous centroids. The algorithm repeats it until the centroids do not move significantly.
The k-means always converges, but is highly dependent on the initialization of number of clusters and centroids vectors, therefore, defining distance from each data point might be a better initialization for centroids than random. To this point, we used a Silhouette analysis to find the separation distance between the resulting clusters. The silhouette illustrates a graphical measure over the distances of each point in clusters to the points in the neighboring clusters.  This enables us to assess and find the best number of clusters visually. Silhouette coefficient has a range of $[-1, 1]$, where $1$ indicates the sample is in the proper cluster and far from the neighboring clusters, and $0$ indicates that the sample is very close to the neighboring boundary of clusters and negative values designate the data point has been assigned to a wrong cluster. Thus by performing a partitioning k-means clustering shows a strong difference in daily load profiles for working days, non-working days and weekends.

# **Smart EVSE Management**
An EVSE can be defined as a flexible and deferrable load, where you can schedule them in a different charge limit and timeslot. The SWTCH load management system ($EViDect^®$) will manage the load profile through the permissible interval and find the best working time slots at a minimum cost. This model can help users to first find the proper time for EVSEs usage and then find the optimized load profile vector for the building accumulated loads, including EVSEs. In this optimization regarding to general model, we have:
$$\min \sum _{t=1}^{24} n x_{t} P_{t}$$
Subject to.
$$\sum _{i=1}^{24} n_t p_{t} - x_{t} = 0 $$
$$c_{min} < c_{t}< c_{max}$$
$$x_{min} < x_{t}< x_{max}$$
$$n_{min} < n_{t}< n_{max}$$
where, $x$ and $n$ are the state variables representing the assigned power and number of EVSE in a location. The $p$ is the TOU energy pricing, and $c$ is the permissible capacity of a site, which is calculated using the k-means analysis. 
The boundaries can be selected using the EVWatts database ([here](https://energetics.com/evwatts-station-dashboard)), where it gives a valid and realistic info for the current utilization time and capacity to us.  We used conEdison TOU pricing for residential and non-residential cases published [here](https://www.coned.com/en/accounts-billing/your-bill/time-of-use).


# About Dataset
The dataset is part of [RTEM Hackathon](https://www.rtemhackathon.com/), where a series of video tutorials are associated with this dataset can be found [here](https://www.kaggle.com/datasets/ponybiam/onboard-api-intro).


## Needed libary and preparation

We used Python 3.8.8 and loaded the following modules to ensure a minimum level of reproducibility while requiring little additional effort. 
```
Module              Version
============================
dateutil            2.8.2
decouple            
gekko               1.0.2
matplotlib          3.5.1
numpy               1.22.2
onboard             
pandas              1.4.1
requests            2.26.0
sklearn             1.0.2
```
You need to `pip install` them first. i.e., `pip install gekko==1.0.2`

In [ ]:
!pip install onboard.client

## solution and case study
The provided solution here is a generic approach applicable to all buildings with a data formated according to RTEM dataset. 
First thing first, load the needed python modules. General python modules which will be used for parsing, timestamping, and data processing: 


In [ ]:
import sys, requests, json, dateutil
from datetime import datetime, timezone, timedelta
import pandas as pd
import numpy as np

and let's take the advantage of provided RTEM APIs using 'onboard' model: 

In [ ]:
from onboard.client import ProductionAPIClient
from onboard.client.dataframes import points_df_from_timeseries
from onboard.client.models import PointSelector, TimeseriesQuery, PointData

We used gekko for optimization:

In [ ]:
from gekko import GEKKO

the modules for clustering and data analytics are:

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE

and to visually see the outcome of your work you need to use:

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.colors
from matplotlib.lines import Line2D
from matplotlib import gridspec
import matplotlib.ticker as plticker

and finally to define environmantal variables and configurations, we suggest using `decouple`:

In [ ]:
from decouple import config

you should make an .env with the following information in it:
```
BUILDING_IDs=all
SAVE_CSV=False
CURTAILMENT=True
ST_TIER_1=01/01/2022
ST_TIER_2=01/07/2022
ST_TIER_3=01/10/2022
RESIDENTIAL_TOU_1=[1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44]
RESIDENTIAL_TOU_2=[1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,25.5,25.5,25.5,25.5,25.5,25.5,25.5,25.5,25.5,25.5,25.5,25.5,25.5,25.5,25.5,25.5]
RESIDENTIAL_TOU_3=[1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44,9.44]
BUSINESS_TOU_1=[1.38,1.38,1.38,1.38,1.38,1.38,1.38,1.38,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62]
BUSINESS_TOU_2=[1.38,1.38,1.38,1.38,1.38,1.38,1.38,1.38,37.82,37.82,37.82,37.82,37.82,37.82,37.82,37.82,37.82,37.82,37.82,37.82,37.82,37.82,37.82,37.82]
BUSINESS_TOU_3=[1.38,1.38,1.38,1.38,1.38,1.38,1.38,1.38,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62,18.62]
ONBOARD_API_KEY=YOUR_RTEM_API_KEY
```
where you can set:
* `BUILDING_IDs` set it to `all` or a building id in a list form i.e.;`[103]`
* `SAVE_CSV` default is `False` and you may change it to `True` if you want to save the csv files
* `CURTAILMENT` by setting it to `True` the curtailment and scheduling load management will be enabled otherwise selecting `False` provides only the load management through the scheduling 
* `ST_TIER_1` is for starting of the first tiered ToU period, i.e. for NY it is `01/01/2022`
* `ST_TIER_2` is for starting the second tiered ToU period, i.e. for NY it is `01/07/2022`
* `ST_TIER_3` is for starting the third tiered ToU period, i.e. for NY it is `01/10/2022`
* `RESIDENTIAL_TOU_1` is the list of hourly energy prices for residential customers in the first tiered ToU period in cents
* `RESIDENTIAL_TOU_2` is the list of hourly energy prices for residential customers in the second tiered ToU period in cents
* `RESIDENTIAL_TOU_3` is the list of hourly energy prices for residential customers in the third tiered ToU period in cents
* `BUSINESS_TOU_1` is the list of hourly energy prices for non-residential customers in the first tiered ToU period in cents
* `BUSINESS_TOU_2` is the list of hourly energy prices for non-residential customers in the second tiered ToU period in cents
* `BUSINESS_TOU_3` is the list of hourly energy prices for non-residential customers in the third tiered ToU period in cents
* `ONBOARD_API_KEY` add your API key (check this [instruction](https://www.kaggle.com/code/ponybiam/01-rtem-onboard-api-and-onboard-api-wrapper))

Then simply you can call those environmental variables in using `decouple.config`: 

In [ ]:
api_key = config('ONBOARD_API_KEY')
save_to_csv = config('SAVE_CSV', default=False, cast=bool)
curtailment = config('CURTAILMENT', default=True, cast=bool)

then we can proceed in creating a client session using the acquired API key via provided onboard client class:

In [ ]:
client = ProductionAPIClient(api_key=api_key)

in order to get the building info and associated points we can use some of the available APIs;

In [ ]:
def building_info(client, building_ids = None):
  if api_key is None:
    print("API key must be set as environment variable ONBOARD_API_KEY")
    sys.exit(1)
  data_poits = []
  bdgs_with_panel = []
  key = { "key": api_key}
  response = requests.post("https://api.onboarddata.io/login/api-key", data=key).json()
  headers = {"Authorization": "Bearer "+ response["access_token"]}
  if building_ids == None:
    bdgs = requests.get("https://api.onboarddata.io/buildings", headers=headers).json()
    building_ids = [pdg.get('id') for pdg in bdgs]

  for bdgs_id in building_ids:
    try:
      res = requests.get(f"https://api.onboarddata.io/buildings/{bdgs_id}", headers=headers)
      if res.status_code == 200:
        res = res.json()
        bdg_type = res.get('info').get('customerType')
        point_count = res.get('point_count')
        equip_count = res.get('equip_count')
        area = res.get('sq_ft')
        query = PointSelector()
        query.equipment_types = ['meter']
        query.buildings = [bdgs_id]
        selection = client.select_points(query)
        points = selection["points"]
        if points:
          bdgs_with_panel.append(bdgs_id)
          for point in points:        
            res = requests.get(f"https://api.onboarddata.io/buildings/{bdgs_id}/points/{point}", headers=headers).json()
            if res['measurement_id'] in [13, 14]:
              data_poits.append({'building': bdgs_id,
                                 'type':bdg_type,
                                 'point': point,
                                 'first_updated': res['first_updated'],
                                 'last_updated': res['last_updated'],
                                 'description': res['description'],
                                 'name': res['name'],
                                 'area': area,
                                 'equip_count': equip_count,
                                 'point_count': point_count})
    except Exception as e:
      print(e)
      pass
  return data_poits, bdgs_with_panel

In the above function if `building_ids` is set to `None` then it explores all available building and collects and find the corresponding `building_id`. Either way, we are interested only in buildings with electrical panels equipped with an electricity meter. To this end, we are focusing only on `measurement_id` 13 and 14 which are related to power and energy metering devices. The output of this function is a dictionary representing the general information of buildings and a list of buildings with an electricity meter (buildings with an electrical panel).

In [ ]:
data_points, bdgs_with_panel = building_info(client, bdgs_ids)

To get time-series data we need to establish a `TimeseriesQuery`. Unfortunately not all the data grouped under `measurment_id` 13 and 14 are relevant to electrical power and we found in some cases it contains other irrelevant data like electricity consumption (with kWh unit), so we considered an additional filter before making a query for historical data.

In [ ]:
date_now = datetime.now()

if data_points != []:
  for data in data_points:
    building_id = data['building']
    building_type = data['type'].lower() if data['type'] else 'commercial'
    building_name = data['name']
    point_id = data['point']
    TOU = select_ToU(date_now, building_type)
    try:
      if (any(x in building_name.lower() for x in ['commercial']) or
          (building_type in ['multifamily'] and not any(x in building_name.lower() for x in ['gas', 'kwh']))):
        timeseries_query = TimeseriesQuery(point_ids = [point_id],
                                           start = datetime.fromtimestamp(data["first_updated"]/1000, timezone.utc),
                                           end = datetime.fromtimestamp(data["last_updated"]/1000, timezone.utc))
        query_results = client.stream_point_timeseries(timeseries_query)

The output of `stream_point_timeseries` functions is a timestamped data series (point) whose corresponding values (`point.values`) will be used later in our data analytical parts. Moreover, we defined the `select_ToU` function to parse and pick the proper ToU pricing:

In [ ]:
def select_ToU(date, building_type):
  start_tier_1 = dateutil.parser.parse(config('ST_TIER_1')).replace(year=datetime.now().year)
  start_tier_2 = dateutil.parser.parse(config('ST_TIER_2')).replace(year=datetime.now().year)
  start_tier_3 = dateutil.parser.parse(config('ST_TIER_3')).replace(year=datetime.now().year)

  if start_tier_1 < date < start_tier_2:
    if building_type in ['multifamily', 'residential']:
      return json.loads(config('RESIDENTIAL_TOU_1'))
    else:
      return json.loads(config('BUSINESS_TOU_1'))
  if start_tier_2 < date < start_tier_3:
    if building_type in ['multifamily', 'residential']:
      return json.loads(config('RESIDENTIAL_TOU_2'))
    else:
      return json.loads(config('BUSINESS_TOU_2'))
  else:
    if building_type in ['multifamily', 'residential']:
      return json.loads(config('RESIDENTIAL_TOU_3'))
    else:
      return json.loads(config('BUSINESS_TOU_3'))

we will use dataframe and pandas for the rest of our data manipulation and analysis, so you can simply perform:

In [ ]:
for point in query_results:
  df = pd.DataFrame(point.values)
  df.columns = point.columns

The provided data needs some prepration to be used in the next steps, we can make and cal `date_preparation` function to do this matter fo us, 

In [ ]:
def date_preparation(df, bulding_name, point_id, print_image = False):
    df['datetime'] = pd.to_datetime(df['time'])
    try:
      df = df.drop(['time', 'clean'], axis=1)
    except Exception as e:
      print(e)
      pass
    df['Month'] = df['datetime'].dt.month
    df['Year'] = df['datetime'].dt.year
    df['Week'] = df['datetime'].dt.isocalendar().week
    Monthly = pd.pivot_table(df,index='Month',columns='Year',values='raw',aggfunc=np.sum)
    Weekly = pd.pivot_table(df,index='Week',columns='Year',values='raw',aggfunc=np.sum)
    df = df.set_index('datetime')
    df = df.replace('?', np.nan)
    df = df.astype(np.float64).fillna(method='bfill')
    df_hourly = df.resample('H').mean()
    df_daily = df.resample('D').sum()
    df_daily_m = df.resample('D').mean()
    df_daily['Date'] = df_daily.index
    df_daily ['Date'] = pd.to_datetime(df_daily['Date'])
    df_daily['Weekday'] = df_daily['Date'].dt.day_name()
    df_daily = df_daily.drop(['Date'], axis=1)
    df_hourly['hour'] = df_hourly.index.hour
    df_hourly.index = df_hourly.index.date
    df_pivot = df_hourly.pivot(columns='hour')
    df_pivot = df_pivot.dropna()
    df_pivot_power = df_pivot['raw'].copy()
    if print_image:
      ax1 = df_pivot_power.T.plot(figsize=(6,4), legend=False, color='black', alpha=0.2)
      ax1.set_xlabel(r"$Time$ (hr)")
      ax1.set_ylabel(r"$Consumption$ (kW)")
      plt.subplots_adjust(left=0.13, right=0.98, top=0.98, bottom=0.13)
      plt.savefig(str(bulding_name) + '_' + str(point_id) +'_consumption')
      plt.clf()
    return df_pivot_power

briefly, the above function cleans the data, resemble it hourly, and pivots it around the 'hour' column. We would use these pivoted dataframes to find the silhouette and the corresponding fit transform.
![load_profile](https://)

In [ ]:
def find_silhouette(df_pivot_power, bulding_name, point_id, print_image = False):
    sillhoute_scores = []
    n_cluster_list = np.arange(2,31).astype(int)
    X = df_pivot_power.values.copy()
    # Very important to scale!
    sc = MinMaxScaler()
    fit_transform = sc.fit_transform(X)
    for n_cluster in n_cluster_list:
      kmeans = KMeans(n_clusters=n_cluster)
      cluster_found = kmeans.fit_predict(X)
      sillhoute_scores.append(silhouette_score(fit_transform, kmeans.labels_))
    if print_image:
      plt.plot(sillhoute_scores, 'k')
      plt.ylabel(r'$Silhouette$')
      plt.xlabel(r'$Cluster$\#')
      plt.title('Silhouette Analysis')
      plt.grid(True)
      plt.subplots_adjust(left=0.2, right=0.98, top=0.98, bottom=0.13)
      plt.savefig(str(bulding_name) + '_' + str(point_id) +'_sillhoute')
      plt.clf()
    return fit_transform

calling
```
df_pivot_power = date_preparation(df, building_id, point_id)
fit_transform = find_silhouette(df_pivot_power, building_id, point_id)
```
will enable us to proceed with the k-means analysis;

![silhouette](http://)

In [ ]:
def find_k_means(df_pivot_power, fit_transform, bulding_name, bulding_type, point_id, TOU, curtailment = True, print_image = False):
    kmeans = KMeans(n_clusters = 5)
    cluster_found = kmeans.fit_predict(fit_transform)
    cluster_found_sr = pd.Series(cluster_found, name='cluster')
    df_pivot_power = df_pivot_power.set_index(cluster_found_sr, append=True)
    cluster_values = sorted(df_pivot_power.index.get_level_values('cluster').unique())
    baseloads = [(list(df_pivot_power.xs(cluster, level=1).median(numeric_only=True))[:24]) for cluster in cluster_values]
    baseload_max = [max(baseload) for baseload in baseloads]
    baseload_cluster = baseload_max.index(max(baseload_max))
    baseload = list(df_pivot_power.xs(baseload_cluster, level=1).median(numeric_only=True))[:24]

    m, C, n, x, cap_low, cap_up = optimal_cp(baseload, TOU, curtailment)

    if m and C and n and x and cap_low and cap_up:
      accumulated_power = [a*b for a,b in zip(list(n.value),list(x.value))] if curtailment else len(m.time)*[x * min(n.value[1:])]
      min(n.value[1:])
      accumulated_cost = sum([a*b for a,b in zip(TOU,accumulated_power)])/100.0
      if print_image:
        fig = plt.figure(figsize=(6.4, 9.2))
        gs  = gridspec.GridSpec(4, 1, height_ratios=[2, 1 ,1, 1])
        ax1 = plt.subplot(gs[0])
        ax2 = plt.subplot(gs[1])
        ax3 = plt.subplot(gs[2])
        ax4 = plt.subplot(gs[3])
        color_list = ['blue', 'red', 'green', 'black', 'gray']
        for cluster, color in zip(cluster_values, color_list):
          df_pivot_power.xs(cluster, level=1).T.plot(ax=ax1, legend=False, alpha=0.01, color=color, label= False)
          df_pivot_power.xs(cluster, level=1).median().plot(ax=ax1, color=color, legend=False, label= False, alpha= 1, ls='--' )
        ax1.set_title(f'Load Profile {bulding_name}_{point_id}',fontsize=10) 
        ax1.set_ylabel(r'Consumption (kW)',fontsize=10)
        lines = [Line2D([0], [0], color=c, linewidth=2, linestyle='--') for c in color_list]
        labels = [f'cluster#{cluster+1}' for cluster in cluster_values]
        ax1.legend(lines, labels)
        ax1.xaxis.set_visible(False)
        ax1.set_xticklabels([])
        ax1.xaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
        ax1.grid('on', which='major', axis='x' )
        ax2.plot(m.time, cap_up,'g-')
        ax2.plot(m.time, cap_low, 'k--')
        ax2.plot(m.time,C.value,'r:')
        ax2.set_title('Capacity',fontsize=10)
        ax2.set_ylim([1,int(max(cap_up)*1.2)])
        ax2.set_ylabel('Power (kW)',fontsize=10)
        ax2.set_xticklabels([])
        ax2.xaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
        ax2.legend(["Upper bound", "Lower bound", "Asgd power"])

        ax3.step(m.time,x.value,'b--') if curtailment else ax3.step(m.time,len(m.time)* [x],'b--')
        ax3.set_title('Chargers Schedule Setpoint',fontsize=10)
        ax3.set_ylabel('Power (kW)',fontsize=10)
        ax3.set_ylim([1,int(max(x.value)*1.2)]) if curtailment else ax3.set_ylim([1,x*1.2])
        ax3.set_xticklabels([])
        ax3.xaxis.set_major_locator(plticker.MultipleLocator(base=1.0))

        ax4.step(m.time,n.value,'r-') if curtailment else ax4.step(m.time,len(m.time)* min(n.value[1:]),'r-')
        ax4.set_title('No. of Chargers',fontsize=10)
        ax4.set_ylim([1,int(max(n.value)*1.2)])
        ax4.set_xlabel('Daytime',fontsize=10)
        ax4.xaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
        ax4.set_xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])
        fig.tight_layout()
        plt.savefig(f'Profile {bulding_type} {bulding_name} {point_id}')
        plt.clf()
        plt.close(fig)
      return baseload, max(n.value), min(n.value[1:]), accumulated_cost
    else:
      return baseload, 0, 0

The above function first finds the clustered daily load profile,  find the most dominated and higher profile, and records it as the baseline load profile ('baseloads'). Then using the baseline we can find the remaining capacity, and optimally select the number of EVSEs and the best schedule to minimize the energy costs;


In [ ]:
def optimal_cp(demand_array, TOU, curtailment = True):
    m = GEKKO(remote=False)
    m.time = np.linspace(0,23,24)
    demand_max = max(demand_array)
    demand_max = demand_max if demand_max % 100 == 0 else demand_max + 100 - demand_max % 100
    cap_low = [demand_max-d for d in demand_array]
    cap_up = 24*[demand_max * 0.8] if max(cap_low) < demand_max * 0.9 else 24*[demand_max]
    cap_low = cap_low if max(cap_low) < max(cap_up) else [i*0.8 for i in cap_low]
    k = 1.1 #Safety factor
    CL = m.Param(value=cap_low)
    CH = m.Param(value=cap_up)
    TOU = m.Param(value=TOU)
    if curtailment == True:
      x = m.MV(lb=3, ub=6.6)
      x.STATUS = 1
    else:
      x = 6.6
    n = m.MV(lb=1, ub = max(cap_low)/6 if max(cap_low)%6!=0 else max(cap_low)/5.95, integer=True)
    n.STATUS = 1
    C = m.SV(value=cap_low[0])
    m.Equations([C>=CL, C<=CH])
    m.Equation(k*C - n* x == 0)
    m.Minimize(TOU*n*x)
    m.options.IMODE = 6
    try:
      m.solve(disp=False,debug=False)
      return m, C, n, x, cap_low, cap_up
    except:
      optimal_cp(demand_array, TOU, curtailment = False)
      return m, C, n, x, cap_low, cap_up
    else:
      print(e)
      return 6 * [None]

![image](https://raw.githubusercontent.com/swtch-energy/rtem/main/docs/img/Profile%20multifamily%20248%20208338.png?token=GHSAT0AAAAAABUX3UHC2TSBG7RPDDL3APNQYUVCGDA)

![image](https://raw.githubusercontent.com/swtch-energy/rtem/main/docs/img/Profile%20multifamily%20200%20203805.png?token=GHSAT0AAAAAABUX3UHDNH7AFV5TOUOVCJESYUVCCWQ)

In [ ]:
from IPython.display import Image
Image("../input/rtem-images/Profile multifamily 200 203805.png")

To generate a summary of available buildings we make a list of dictionaries and update the building data with maximum load, number of EVSEs, and the associated energy costs;

In [ ]:
data.update({'max_demand': max(baseload), 'Charger_num': int(max_cp), 'Cost ($/day)': accumulated_cost})

and save it as a `csv` file;

In [ ]:
def save_building_table(cp_numbers, curtailment):
    df = pd.DataFrame(data_points)
    df = df.drop(['first_updated', 'last_updated', 'name'], axis=1)
    df.to_csv('building_info_slms.csv', sep=',',
              encoding='utf-8') if curtailment else df.to_csv('building_info_lms.csv', 
                                                              sep=',', encoding='utf-8')

The full code of this excerise can be found here.